In [67]:
import numpy as np

#기본 가정
B = 50 # batch size (EA/Batch)
SHIFT = 8 # shift duration in hours
TH_target_unit = 1000 # panels/shift duraton 
TH_target_batch = TH_tatget_unit/B/SHIFT # panels/shift duraton 

#공정 모델
#[:,0] : part processing time (parts/hr)
#[:,1] : CV of part processing time
#[:,2] : unit price of facility
#[:,3] : number of facility or process

param_process = np.array([[42, 2, 50, 3],[42, 2, 50, 3], [25, 1, 100, 6], [50, 0.75, 20, 5]])

In [68]:
# 각 공정의 단위 제품 effective time 계산
te = np.zeros(4)
te = 1/param_process[:, 0]

#각 공정의 배치 제품 effective time 계산
tb = np.zeros(4)
tb = te * B

#각 공정 생산 시간의 변동성 (SCV: Squared Coefficient of Variability)
cb_2 = np.zeros(4)
cb_2 = np.power(param_process[:, 1],2)/B

#각 공정의 가동률 (utilization)
u = np.zeros(4)
u = TH_target_batch * tb / param_process[:, 3]

In [69]:
ca_2 = np.zeros(4)
cd_2 = np.zeros(4)

#전 공정의 출발률 변동성(cd_2[i]) = 후 공정의 도착률 변동성(ca_2[i+1])
cd_2[0] = 1 + (1-np.power(u[0],2))*(ca_2[0]-1) + (np.power(u[0],2)/np.sqrt(param_process[0,3])) * (cb_2[0]-1)
ca_2[1] = cd_2[0]
cd_2[1] = 1 + (1-np.power(u[1],2))*(ca_2[1]-1) + (np.power(u[1],2)/np.sqrt(param_process[1,3])) * (cb_2[1]-1)
ca_2[2] = cd_2[1]
cd_2[2] = 1 + (1-np.power(u[2],2))*(ca_2[2]-1) + (np.power(u[2],2)/np.sqrt(param_process[2,3])) * (cb_2[2]-1)
ca_2[3] = cd_2[2]
cd_2[3] = 1 + (1-np.power(u[3],2))*(ca_2[3]-1) + (np.power(u[3],2)/np.sqrt(param_process[3,3])) * (cb_2[3]-1)

In [70]:
CT = np.zeros(4)

CT = [(ca_2 + cb_2)/2] * np.power(u,np.sqrt(2*(param_process[:, 3]+1))-1)/(param_process[:, 3]*(1-u))* tb + tb

In [71]:
CT_tot = np.sum(CT)

In [76]:
WIP_q = (CT-tb)*TH_target_batch

In [77]:
WIP_q = np.ceil(WIP_q)

In [78]:
WIP_q

array([[ 5., 34.,  1.,  1.]])